In [4]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import igraph as ig

print('liste des fichiers dans le répertoire local:', os.listdir()) #lister les fichiers du répertoire local

path='/home/ibotcazou/Bureau/Master_data_science/DATAS_M2/Option Biologie/projet'

#Chargement des données et Preprocessing
try:
    data = pd.read_csv(path + '/4x_sans_dosage.csv'
                       ,header=None,sep=';')
except:
    print("Le fichier 4x_sans_dosage.csv n'est pas dans le bon répertoire")

data



liste des fichiers dans le répertoire local: ['projet_2X.Rmd', 'tétaploïde_avec_dosage.ipynb', 'donnees_tétaploïde.nb.html', '.ipynb_checkpoints', '.RData', 'DataGeno2x_simple.ipynb', 'Avec_dosage_4X.ipynb', 'pdf', 'README.txt', 'Brouillon_généalogie_rosier.ipynb', 'Pop2_v2.ipynb', '.Rhistory']


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,10,0,0,0,100,150,160,170,200,240,220,210,300,0,0,0,1,1
1,10,30,0,0,110,0,0,0,210,220,230,210,311,331,301,0,1,2
2,30,40,50,70,100,120,0,0,210,0,0,0,300,350,0,0,1,3
3,20,50,60,0,130,140,0,0,250,0,0,0,321,341,351,371,1,4
4,30,0,0,0,100,110,150,0,240,250,0,0,310,370,330,0,1,5
5,10,30,60,0,110,150,170,0,220,240,0,0,300,330,370,0,2,6
6,50,10,60,0,130,140,160,170,250,240,200,0,340,370,300,0,2,7
7,20,30,60,0,140,130,100,0,250,0,0,0,310,320,330,340,3,8
8,50,60,30,0,130,140,150,170,220,250,240,0,300,320,340,370,3,9
9,30,70,50,0,100,110,120,150,240,210,220,0,300,330,350,370,3,10


In [5]:
# Définir la colonne '18' comme indice
data = data.set_index(data.columns[-1])

data.index.name = 'index'

nouveaux_noms = {}
# Créez un dictionnaire de correspondance des noms de colonnes

for i in range(16):
    nouveaux_noms[i] = f'marqueur_{i//4 +1}.{i%4}'
    # i correspond à l'ancien nom des colonnes 
    
    
nouveaux_noms[16] = 'generation'

data.rename(columns=nouveaux_noms,inplace=True)

n = len(data)
car = 4 #nb de marqueurs
gen = 6 # on ne veut que pour les génération plus grande que 1 


data

,marqueur_1.0,marqueur_1.1,marqueur_1.2,marqueur_1.3,marqueur_2.0,marqueur_2.1,marqueur_2.2,marqueur_2.3,marqueur_3.0,marqueur_3.1,marqueur_3.2,marqueur_3.3,marqueur_4.0,marqueur_4.1,marqueur_4.2,marqueur_4.3,generation
index,,,,,,,,,,,,,,,,,
1,10,0,0,0,100,150,160,170,200,240,220,210,300,0,0,0,1
2,10,30,0,0,110,0,0,0,210,220,230,210,311,331,301,0,1
3,30,40,50,70,100,120,0,0,210,0,0,0,300,350,0,0,1
4,20,50,60,0,130,140,0,0,250,0,0,0,321,341,351,371,1
5,30,0,0,0,100,110,150,0,240,250,0,0,310,370,330,0,1
6,10,30,60,0,110,150,170,0,220,240,0,0,300,330,370,0,2
7,50,10,60,0,130,140,160,170,250,240,200,0,340,370,300,0,2
8,20,30,60,0,140,130,100,0,250,0,0,0,310,320,330,340,3
9,50,60,30,0,130,140,150,170,220,250,240,0,300,320,340,370,3
